In [1]:
import cv2
import numpy as np
import os
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, UpSampling2D
from skimage.metrics import peak_signal_noise_ratio as psnr
from skimage.metrics import structural_similarity as ssim
import pywt
from PyQt5.QtWidgets import QApplication, QWidget, QPushButton, QLabel, QFileDialog, QVBoxLayout, QHBoxLayout
from PyQt5.QtGui import QPixmap, QImage
from PyQt5.QtCore import Qt
import sys

In [2]:
# Load and preprocess clean images
def load_clean_image(image_path, size=(128, 128)):
    """Load an image and normalize it to [0, 1]."""
    img = load_img(image_path, target_size=size)
    img_array = img_to_array(img) / 255.0
    return img_array

In [3]:
# Add Gaussian noise
def add_gaussian_noise(image_array, sigma=0.1):
    """Add Gaussian noise to an image."""
    noise = np.random.normal(0, sigma, image_array.shape)
    noisy_image = np.clip(image_array + noise, 0, 1)
    return noisy_image

In [4]:
# Wavelet-based denoising
def wavelet_denoising(image, wavelet='db1', level=1):
    """Apply wavelet-based denoising to an RGB image by processing each channel separately."""
    # Initialize output image
    denoised_image = np.zeros_like(image)
    
    # Process each channel (R, G, B) independently
    for channel in range(image.shape[2]):
        # Perform wavelet decomposition on the single channel
        coeffs = pywt.wavedec2(image[:, :, channel], wavelet, level=level)
        
        # Estimate noise level and set threshold
        detail_coeffs = coeffs[-1]
        if isinstance(detail_coeffs, tuple):
            all_details = np.concatenate([c.flatten() for c in detail_coeffs])
        else:
            all_details = detail_coeffs.flatten()
        threshold = np.std(all_details) * np.sqrt(2 * np.log(image.size // image.shape[2]))
        
        # Threshold the detail coefficients while preserving structure
        thresholded_coeffs = [coeffs[0]]
        for detail in coeffs[1:]:
            if isinstance(detail, tuple):
                thresholded_detail = tuple(pywt.threshold(c, threshold, mode='soft') for c in detail)
            else:
                thresholded_detail = pywt.threshold(detail, mode='soft')
            thresholded_coeffs.append(thresholded_detail)
        
        # Reconstruct the denoised channel
        denoised_channel = pywt.waverec2(thresholded_coeffs, wavelet)
        denoised_image[:, :, channel] = np.clip(denoised_channel, 0, 1)
    
    return denoised_image

In [5]:
# Define and train a simple convolutional autoencoder
def build_autoencoder(input_shape=(128, 128, 3)):
    """Build a convolutional autoencoder for denoising."""
    input_img = Input(shape=input_shape)
    # Encoder
    x = Conv2D(32, (3, 3), activation='relu', padding='same')(input_img)
    x = MaxPooling2D((2, 2), padding='same')(x)
    x = Conv2D(32, (3, 3), activation='relu', padding='same')(x)
    encoded = MaxPooling2D((2, 2), padding='same')(x)
    # Decoder
    x = Conv2D(32, (3, 3), activation='relu', padding='same')(encoded)
    x = UpSampling2D((2, 2))(x)
    x = Conv2D(32, (3, 3), activation='relu', padding='same')(x)
    x = UpSampling2D((2, 2))(x)
    decoded = Conv2D(3, (3, 3), activation='sigmoid', padding='same')(x)
    autoencoder = Model(input_img, decoded)
    autoencoder.compile(optimizer='adam', loss='mse')
    return autoencoder

In [6]:
# Denoise image using the autoencoder
def denoise_image(autoencoder, noisy_image):
    """Denoise an image using the trained autoencoder."""
    noisy_image = np.expand_dims(noisy_image, axis=0)
    denoised_image = autoencoder.predict(noisy_image, verbose=0)[0]
    return denoised_image

In [7]:
# Convert numpy array to QImage for PyQt5 display
def numpy_to_qimage(image):
    """Convert a numpy image array to QImage."""
    image = (image * 255).astype(np.uint8)
    if len(image.shape) == 3:
        height, width, channel = image.shape
        return QImage(image.data, width, height, 3 * width, QImage.Format_RGB888).rgbSwapped()
    else:
        height, width = image.shape
        return QImage(image.data, width, height, width, QImage.Format_Grayscale8)

In [8]:
# Compute evaluation metrics
def compute_metrics(original, denoised):
    """Compute PSNR, SSIM, and MSE between original and denoised images."""
    mse = np.mean((original - denoised) ** 2)
    psnr_value = psnr(original, denoised, data_range=1.0)
    ssim_value = ssim(original, denoised, channel_axis=2, data_range=1.0)
    return mse, psnr_value, ssim_value

In [9]:
# PyQt5 Application
class NoiseReductionApp(QWidget):
    def __init__(self, autoencoder):
        super().__init__()
        self.autoencoder = autoencoder
        self.initUI()

    def initUI(self):
        """Initialize the GUI layout."""
        self.setWindowTitle("Noise Reduction Comparison Tool")
        self.setGeometry(100, 100, 1200, 600)

        layout = QVBoxLayout()
        
        # Upload button
        self.upload_btn = QPushButton("Upload Image", self)
        self.upload_btn.clicked.connect(self.upload_image)
        layout.addWidget(self.upload_btn)

        # Labels for displaying images
        self.image_labels = {
            "original": QLabel(self),
            "noisy": QLabel(self),
            "gaussian": QLabel(self),
            "median": QLabel(self),
            "wavelet": QLabel(self),
            "autoencoder": QLabel(self)
        }
        image_layout = QHBoxLayout()
        for key, label in self.image_labels.items():
            label.setAlignment(Qt.AlignCenter)
            label_layout = QVBoxLayout()
            label_layout.addWidget(QLabel(key.capitalize()))
            label_layout.addWidget(label)
            image_layout.addLayout(label_layout)
        layout.addLayout(image_layout)

        # Metrics display
        self.metrics_label = QLabel("Metrics: Not computed", self)
        layout.addWidget(self.metrics_label)

        self.setLayout(layout)

    def upload_image(self):
        """Handle image upload and denoising."""
        file_path, _ = QFileDialog.getOpenFileName(self, "Select Image", "", "Image Files (*.png *.jpg *.jpeg)")
        if file_path:
            # Load and preprocess image
            image = load_clean_image(file_path)
            if image is None:
                return
            
            noisy_image = add_gaussian_noise(image)

            # Apply denoising techniques
            gaussian_denoised = cv2.GaussianBlur(noisy_image, (5, 5), 0)
            median_denoised = cv2.medianBlur((noisy_image * 255).astype(np.uint8), 5) / 255.0
            wavelet_denoised = wavelet_denoising(noisy_image)
            autoencoder_denoised = denoise_image(self.autoencoder, noisy_image)

            # Display images
            self.image_labels["original"].setPixmap(QPixmap.fromImage(numpy_to_qimage(image)))
            self.image_labels["noisy"].setPixmap(QPixmap.fromImage(numpy_to_qimage(noisy_image)))
            self.image_labels["gaussian"].setPixmap(QPixmap.fromImage(numpy_to_qimage(gaussian_denoised)))
            self.image_labels["median"].setPixmap(QPixmap.fromImage(numpy_to_qimage(median_denoised)))
            self.image_labels["wavelet"].setPixmap(QPixmap.fromImage(numpy_to_qimage(wavelet_denoised)))
            self.image_labels["autoencoder"].setPixmap(QPixmap.fromImage(numpy_to_qimage(autoencoder_denoised)))

            # Compute and display metrics
            metrics = {
                "Gaussian": compute_metrics(image, gaussian_denoised),
                "Median": compute_metrics(image, median_denoised),
                "Wavelet": compute_metrics(image, wavelet_denoised),
                "Autoencoder": compute_metrics(image, autoencoder_denoised)
            }
            metrics_text = "\n".join([f"{key}: MSE={m[0]:.4f}, PSNR={m[1]:.2f}, SSIM={m[2]:.4f}" for key, m in metrics.items()])
            self.metrics_label.setText(f"Metrics:\n{metrics_text}")

In [10]:
# Load dataset (example: use a small subset for training)
dataset_path = "C://Users/user/OneDrive/Documents/SRM-Classes/Sem4/20PCSC55J-ComputerVision/CT3Dataset"  
clean_images = []
noisy_images = []
if not os.path.exists(dataset_path):
    logging.error(f"Dataset path {dataset_path} does not exist")
    sys.exit(1)

for filename in os.listdir(dataset_path)[:100]:  # Limit to 100 images for quick training
    image_array = load_clean_image(os.path.join(dataset_path, filename))
    if image_array is not None:
        clean_images.append(image_array)
        noisy_images.append(add_gaussian_noise(image_array))

if not clean_images:
    logging.error("No valid images loaded from dataset")
    sys.exit(1)

clean_images = np.array(clean_images)
noisy_images = np.array(noisy_images)

# Build and train autoencoder
autoencoder = build_autoencoder()
if autoencoder is None:
    logging.error("Failed to build autoencoder")
    sys.exit(1)

autoencoder.fit(noisy_images, clean_images, epochs=10, batch_size=32, validation_split=0.2, verbose=1)

# Run PyQt5 application
app = QApplication(sys.argv)
window = NoiseReductionApp(autoencoder)
window.show()
sys.exit(app.exec_())

Epoch 1/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 3s 3s/step - loss: 0.1146 - val_loss: 0.1163
Epoch 2/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 103ms/step - loss: 0.1090 - val_loss: 0.1094
Epoch 3/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 103ms/step - loss: 0.1039 - val_loss: 0.1025
Epoch 4/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 101ms/step - loss: 0.0989 - val_loss: 0.0959
Epoch 5/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 112ms/step - loss: 0.0942 - val_loss: 0.0909
Epoch 6/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 106ms/step - loss: 0.0907 - val_loss: 0.0874
Epoch 7/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 108ms/step - loss: 0.0880 - val_loss: 0.0847
Epoch 8/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 103ms/step - loss: 0.0849 - val_loss: 0.0826
Epoch 9/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 99ms/step - loss: 0.0815 - val_loss: 0.0813
Epoch 10/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 101ms/step - loss: 0.0780 - val_loss: 0.0805


SystemExit: 0

C:\Users\user\anaconda3\Lib\site-packages\IPython\core\interactiveshell.py:3585: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
